In [1]:
import pandas as pd
from gensim.models import word2vec
import artm
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk import stem
from nltk import WordNetLemmatizer
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import xgboost
from sklearn.linear_model import LogisticRegression
import datetime
pd.options.display.max_colwidth=200
import scipy
import time

import telegram

def send_to_telegram(text):

    """Send appropriate links to telegram channel"""

    bot = telegram.Bot(token='379005601:AAH1rv3ESXLWTXbn14gnCxW52eeKc4qnw50')
    # chat_id = -1001111732295
    chat_id = 169719023
    bot.send_message(chat_id=chat_id, text=text)
    time.sleep(15)

def No_with_word(token_text):

    """Concating NO with words"""

    tmp=''
    for i,word in enumerate(token_text):
        if word==u'не':
            tmp+=("_".join(token_text[i:i+2]))
            tmp+= ' '
        else:
            if token_text[i-1]!=u'не':
                tmp+=word
                tmp+=' '
    return tmp


def wrk_words_wt_no(sent):

    """Making lemmatisation for Russian and English text"""

    words=word_tokenize(basic_cleaning2(sent).lower())
    lemmatizer = WordNetLemmatizer()
    try:
        arr=[]
        for i in range(len(words)):
            arr.append(morph.parse(words[i])[0].normal_form)
        arr2 = []
        for i in arr:
            arr2.append(lemmatizer.lemmatize(i, pos='v'))
        arr3 = []
        for i in arr2:
            arr3.append(lemmatizer.lemmatize(i, pos='n'))
        arr4 = []
        for i in arr3:
            arr4.append(lemmatizer.lemmatize(i, pos='a'))
        words1=[w for w in arr4 if w not in stop]
#         words1=[w for w in arr4 if w not in english_stops]
        words1=No_with_word(words1)
        return words1
    except TypeError:
        pass

def basic_cleaning2(string):

    """Cleaning text from numbers and punctuation"""

    string = string.lower()
    string = re.sub('[0-9\(\)\!\^\%\$\'\"\.;,-\?\{\}\[\]\\/]', ' ', string)
    string = re.sub(' +', ' ', string)
    return string





def make_bow(col):

    """Make bow from train_test"""

    binVectorizer = CountVectorizer(binary=True, ngram_range=(1, 1), min_df = 2, max_df=20000, max_features=4000)
#     binVectorizer = CountVectorizer(binary=True, ngram_range=(1, 1), min_df = 2)

    counts = binVectorizer.fit_transform(np.array(collection))
    bow_df = pd.DataFrame(counts.toarray(), columns=binVectorizer.get_feature_names())
    to_include = []
    for i in bow_df.columns:
        if bow_df[i].sum()>=2:
            to_include.append(i)
    bow_df = bow_df[to_include]
#     bow_df = scipy.sparse.csr_matrix(bow_df.values)
    return bow_df




def review_to_wordlist(review):

    """Convert collection to wordlist"""

    words = review.lower().split()
    words = [w for w in words]
    return(words)

def make_w2v(collection):

    """Make w2v model"""

    collection_splited = pd.DataFrame(collection).apply(lambda x:x.apply(review_to_wordlist))
    model = word2vec.Word2Vec(collection_splited[0], size=300, window=10, workers=4, min_count=2)
    w2v = dict(zip(model.wv.index2word, model.wv.syn0))
    return w2v, model

def make_w2v_mean(collection):

    """Mean transformation"""

    collection_df = pd.DataFrame(collection).apply(lambda x:x.apply(review_to_wordlist))
#     collection_df = pd.DataFrame(collection)
    data_mean=mean_vectorizer(w2v).fit(collection_df[0]).transform(collection_df[0])
    w2v_mean = pd.DataFrame(data_mean).reset_index(drop=True)
    return w2v_mean

def make_w2v_tfidf(collection):

    """Tf-idf transformation"""

    collection_df = pd.DataFrame(collection).apply(lambda x:x.apply(review_to_wordlist))
    data_tfidf=tfidf_vectorizer(w2v).fit(collection_df[0]).transform(collection_df[0])
    w2v_tfidf = pd.DataFrame(data_tfidf).reset_index(drop=True)
    return w2v_tfidf

class mean_vectorizer(object):

    """Preprocess word embedding using mean transformation"""

    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

class tfidf_vectorizer(object):

    """Class to preprocess word embedding using tf-idf"""

    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

def make_artm(col):

    """Get artm theta matrixes"""

    collection_train = pd.DataFrame(collection).iloc[train_index].reset_index()
    collection_test = pd.DataFrame(collection).iloc[test_index].reset_index()
    le=LabelEncoder()
    y_transormed = le.fit_transform(df_y)

    arr=[]
    for index_number, i, c in zip(collection_train['index'], collection_train[0], y_transormed):
        arr.append(str(index_number) + ' |@default_class ' + str(i) + ' |@labels_class ' + str(c))

    arr_test=[]
    for index_number, i in zip(collection_test['index'], collection_test[0]):
        arr_test.append(str(index_number) +  ' |@default_class ' + str(i))

    pd.DataFrame(arr,index=None).to_csv('leaver_vw_form.txt',sep='\t',encoding='UTF-8',index=False, header=None)
    pd.DataFrame(arr_test,index=None).to_csv('leaver_vw_form_test.txt',sep='\t',encoding='UTF-8',index=False, header=None)

    batch_vectorizer = artm.BatchVectorizer(data_path="leaver_vw_form.txt", data_format="vowpal_wabbit", target_folder="leaver_vw_form_train", batch_size=100)
    batch_vectorizer_test = artm.BatchVectorizer(data_path="leaver_vw_form_test.txt", data_format="vowpal_wabbit", target_folder="leaver_vw_form_test", batch_size=100)

    T = pd.DataFrame(df_y)[target].nunique()
    print ("количество тем составляет - {}".format(T))# количество тем
    topic_names=["sbj"+str(i) for i in range(T)]

    model_artm = artm.ARTM(num_topics=T,
                           topic_names=topic_names,
                           class_ids={'@default_class':1, '@labels_class':700},
                           num_document_passes=10,
                           seed=79,
                           reuse_theta=True,
                           cache_theta=True,
                           scores=[artm.TopTokensScore(name='top_tokens_score', num_tokens=30, class_id='@default_class')],
                           regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)])

    dictionary=artm.Dictionary(name='dictionary')
    dictionary.gather(batch_vectorizer.data_path)

    model_artm.initialize('dictionary')

    dictionary.filter(min_tf=2,  min_df_rate=0.01)

    model_artm.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score',class_id='@labels_class'))
    model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_def',class_ids=['@default_class']))
    model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_lab',class_ids=['@labels_class']))

    model_artm.scores.add(artm.PerplexityScore(name='PerplexityScore',dictionary='dictionary'))
    model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

    test_transformed = model_artm.transform(batch_vectorizer_test, predict_class_id ='@labels_class').T
    train_transformed = model_artm.transform(batch_vectorizer, predict_class_id ='@labels_class').T

    test_transformed = test_transformed.reset_index().sort_values('index')
    test_transformed = test_transformed.reset_index(drop=True)
    del test_transformed['index']
    test_transformed = test_transformed[sorted(test_transformed.columns)]
    train_transformed = train_transformed.reset_index().sort_values('index')
    del train_transformed['index']
    train_transformed = train_transformed[sorted(train_transformed.columns)]
    train_transformed = train_transformed.reset_index(drop=True)
    artm_transformed = pd.concat([train_transformed, test_transformed], axis=0).reset_index(drop=True)

    return artm_transformed

def train_xgboost(train, y, test, proba):

    """Train and predict using xgboost"""

    clf_xgboost = xgboost.XGBClassifier(n_jobs=-1)
    clf_xgboost = clf_xgboost.fit(train, y)
    if proba == 'probability':

        preds = clf_xgboost.predict_proba(test)

        labels2idx = {label: i for i, label in enumerate(clf_xgboost.classes_)}
        preds = np.array(preds)
        sub = pd.DataFrame()
        for label in clf_xgboost.classes_:
            sub[label] = preds[:, labels2idx[label]]
        preds = sub
    else:
        preds = clf_xgboost.predict(test)

    return preds


def train_mlp(train, y, test, proba):

    """Train neural network"""

    clf_mlp = multilayer_perceptron.MLPClassifier(hidden_layer_sizes=(128,64,),
                                        activation='relu',
                                        solver='adam',
                                        alpha=0.0001,
                                        batch_size=64,
                                        learning_rate='constant',
                                        learning_rate_init=0.001,
                                        power_t=0.5,
                                        max_iter=20,
                                        shuffle=True,
                                        random_state=None,
                                        tol=0.0001,
                                        verbose=False,
                                        warm_start=False,
                                        momentum=0.9,
                                        nesterovs_momentum=True,
                                        early_stopping=True,
                                        validation_fraction=0.1,
                                        beta_1=0.9,
                                        beta_2=0.999, epsilon=1e-08)

    clf_mlp = clf_mlp.fit(train, y)
    if proba == 'probability':

        preds = clf_mlp.predict_proba(test)

        labels2idx = {label: i for i, label in enumerate(clf_mlp.classes_)}
        preds = np.array(preds)
        sub = pd.DataFrame()
        for label in clf_mlp.classes_:
            sub[label] = preds[:, labels2idx[label]]
        preds = sub
    else:
        preds = clf_mlp.predict(test)

    return preds

def train_lr(train, y, test, proba):

    """Train and predict using logistic regression"""

    clf_lr = LogisticRegression(solver="liblinear",
                                multi_class="ovr")
    clf_lr = clf_lr.fit(train, y)
    if proba == 'probability':

        preds = clf_lr.predict_proba(test)

        labels2idx = {label: i for i, label in enumerate(clf_lr.classes_)}
        preds = np.array(preds)
        sub = pd.DataFrame()
        for label in clf_lr.classes_:
            sub[label] = preds[:, labels2idx[label]]
        preds = sub

    else:
        preds = clf_lr.predict(test)

    return preds

def make_cross_val(train, y, clf_custom):

    """Show quality on cross-validation"""

    clf = clf_custom
    cv_scores = []
    folds=3
    from sklearn.cross_validation import KFold
    kf = KFold(len(y), n_folds=folds, shuffle=True, random_state=2016)
    for i, (train_, test_) in enumerate(kf):
        dev_X, val_X = train.ix[train_], train.ix[test_]
        dev_y, val_y = y.ix[train_], y.ix[test_]
        preds = clf(dev_X, dev_y, val_X, 'actual')
        cv_scores.append(accuracy_score(preds, val_y))
    # print ('mean quality of{}'.format(clf), (np.mean(cv_scores)))

def mix_predictions_curtesian(preds_all, list_of_indexes):

    """Finding best combination of predictions"""

    ### make cartesian product
    all_list = []
    for i in combinations_with_replacement(list_of_indexes, 2):
        if i[0] !=i[1]:
            all_list.append(list(i))
    for list_of_indexes in all_list:
        for num_val, i in enumerate(list_of_indexes):
            preds_temp = preds_all[preds_all['clf']==i]
            del preds_temp['clf']
            if num_val == 0:
                preds_first=preds_temp.values
            else:
                preds_next = preds_temp.values
                preds_first+=preds_next
        result = pd.DataFrame(preds_first/len(list_of_indexes))
        today_time = datetime.datetime.now()
        result['clf'] = str(today_time.hour) + '_' + str(today_time.minute) + '_' +  str(today_time.second) + '_' +  str(today_time.microsecond)
        result.columns = preds_all.columns
        preds_all = pd.concat([preds_all, result], axis=0)
    return preds_all

def mix_predictions(preds_all, list_of_indexes):

    """Average blending of appropriate predictions"""

    for num_val, i in enumerate(list_of_indexes):
        preds_temp = preds_all[preds_all['clf']==i]
        del preds_temp['clf']
        if num_val == 0:
            preds_first=preds_temp.values
        else:
            preds_next = preds_temp.values
            preds_first+=preds_next
    result = pd.DataFrame(preds_first/len(list_of_indexes))
    today_time = datetime.datetime.now()
    result['clf'] = str(today_time.hour) + '_' + str(today_time.minute) + '_' +  str(today_time.second) + '_' +  str(today_time.microsecond)
    result.columns = preds_all.columns
    preds_all = pd.concat([preds_all, result], axis=0)
    return preds_all

def make_submission (preds_all, source_test):

    """Takes top-3 probabilities and concates with source data"""

    temp_value = preds_all['clf']
    preds_all= preds_all.loc[:,preds_all.columns !='clf']

    max_3_all =[]
    for i in range(len(preds_all)):
        nums = preds_all.iloc[i].values
        max_3 = heapq.nlargest(3, nums)
        max_3_all.append(max_3)

    preds_all = pd.concat([preds_all.reset_index(drop=True), pd.DataFrame(max_3_all)], axis=1)
    cols = preds_all.iloc[:,:preds_all.shape[1]-3].columns
    target_cols=[0,1,2]

    for i in cols:
        for target_col in target_cols:
            preds_all.loc[preds_all[i]==preds_all[target_col], 'max_value{}'.format(target_col)] = i
    preds_all['clf'] = temp_value.values

    for i in preds_all['clf'].unique():
        submission = pd.concat([preds_all[preds_all['clf']==i].reset_index(drop=True), source_test],axis=1)
        try:
            submission['score_true'] = (submission['max_value0']==submission[target])
            submission['score_true'] = submission['score_true'].astype(int)
            score_true = submission['score_true'].sum()
            print (i)
            print (score_true/len(submission))
        except:
            pass
    submission.to_json('../output/promiss/submisssion{}.json'.format(i))
    return submission

def split_test (path_test):

    """If test shape is higher than  60l script splits into 25k batches"""

    test_to_split = pd.read_excel(path_test)
    num_parts = int(round(test_to_split.shape[0]/25000))
    batch_size = int(test_to_split.shape[0]/num_parts)
    test_to_split.shape[0]/7
    ### нарезка файлов
    max_index=0
    for i in range(int(num_parts)):
        part = test_to_split.iloc[max_index:(max_index+batch_size)]
        max_index = part.index.max()+1
        part.to_excel('input/10_albo_test_{}.xlsx'.format(i), index=False)
        print (max_index)

    all_dfs_input = pd.DataFrame()
    for i in range(int(num_parts)):
        temp = pd.read_excel('input/10_albo_test_{}.xlsx'.format(i))
        all_dfs_input = pd.concat([all_dfs_input, temp],axis=0)

    all_dfs_input.to_excel('temp_input3.xlsx', index=False)

import random

def reduce_train(paths_train):

    """If class count is higher than 3000 script cuts all samples above 3000"""

    pre_train = pd.read_excel(path_train)
    pre_train = pre_train.rename(columns={u'Текст письма': u'Суть обращения', u'Процесс 1' : u'Процесс'})
    pre_train[u'Суть обращения'] = pre_train[u'Категория'].astype(str).fillna(0).replace('nan', '') + ' ' + pre_train[u'Суть обращения'].astype(str)
    pre_train = pre_train[[u'Суть обращения', u'Процесс']]

    cut_df = pd.DataFrame()
    for i in pre_train[u'Процесс'].unique():
        temp = pre_train[pre_train[u'Процесс'] == i]
        if temp.shape[0]>3000:
            temp = temp[:3000]
        else:
            temp = temp
        cut_df = pd.concat([cut_df, temp], axis=0)

    cut_df.to_excel('input/train_albo_v4.xlsx', index=False)

def reduce_train_prob (paths_train):
    all_trains = pd.DataFrame()
    for i in paths_train:
        temp_df = pd.read_excel(i)
        temp_df = temp_df.rename(columns = {u'Категория_CJM':u'Процесс 1', u'Проблематика_Лиза':u'Проблема'})
        temp_df = temp_df[[u'Категория', u'Тема письма', u'Текст письма',u'Процесс 1', u'Проблема']]
        all_trains = pd.concat([all_trains, temp_df],axis=0)


    all_trains[u'Категория'] = all_trains[u'Категория'].apply(lambda x : str(x).replace(u'Стандарт', ''))
    all_trains[u'Суть обращения'] = all_trains[u'Категория'].astype(str)  + ' ' + all_trains[u'Тема письма'].astype(str)+ ' '+ all_trains[u'Текст письма'] .astype(str)
    all_trains['count'] = all_trains[u'Суть обращения'].apply(lambda x : len(x))
    all_trains = all_trains[all_trains['count']>15]
    all_trains[u'Процесс'] = all_trains[u'Процесс 1'] + '_' + all_trains[u'Проблема']
    all_trains = all_trains[[u'Суть обращения', u'Процесс']]
    cut_df = pd.DataFrame()
    for i in all_trains[u'Процесс'].unique():
        temp = all_trains[all_trains[u'Процесс'] == i]
        if temp.shape[0]>1500:
            temp = temp.sample(1500)
        elif temp.shape[0]<50:
            temp = pd.DataFrame()
        else:
            temp = temp
        cut_df = pd.concat([cut_df, temp], axis=0)
#         cut_df.to_excel('input/train_albo_v4_prob.xlsx', index=False)
    return cut_df

def delete_operator(path_sent):
    chat = pd.read_excel(path_sent)
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(lambda x : x+'>>>')
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(lambda x : x.replace('operator:', '<<<'))
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(lambda x : x.replace('client:', '>>>'))
    def replace_operator (col):
        return re.sub('<<<[^>]+>>>', '', col)
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(replace_operator)
    chat.to_excel('input/10_chat_v2_sent.xlsx')

/root/nexar/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%%time
# train = pd.read_csv('../input/toxic/train.csv', nrows=2000)
# test = pd.read_csv('../input/toxic/test.csv', nrows=2000)
train = pd.read_csv('../input/toxic/train.csv')
test = pd.read_csv('../input/toxic/test.csv')
# print (train.shape[0])
# print (test.shape[0])
cols = train.columns
# for i in cols[2:]:
#     print (i, train[i].sum())
sum_of_all = []
for ind in range(train.shape[0]):
    temp = train.iloc[ind]
    sum_of_all.append(temp[cols[2:]].sum())
train['all'] = sum_of_all
cols = train.columns
summing=0
for i in cols[2:]:
#     print (i, train[i].sum())
    if i != 'all':
        summing+=train[i].sum()
hot_df = train[train['all']!=0]
cols = train.columns[2:]
for i in cols:
    test[i]=0
train['train_test'] = 1
test['train_test'] = 0
train_test = pd.concat([train, test])
train_test = train_test.reset_index(drop=True)
text = train_test['comment_text'][0]
# porter_stemmer = PorterStemmer()
# porter_stemmer = stem.porter.PorterStemmer()
stemmer=PorterStemmer()
morph = MorphAnalyzer()
stop = stopwords.words('english')
train_test['comment_text_proc'] = train_test['comment_text'].apply(wrk_words_wt_no)
train_test = train_test.rename(columns={'comment_text_proc':0})


CPU times: user 10min 37s, sys: 1.62 s, total: 10min 39s
Wall time: 10min 38s


In [12]:
# train_test.to_json('../input/toxic/train_test.json')

In [33]:
%%time
train_test=pd.read_json('../input/toxic/train_test.json')

CPU times: user 3.82 s, sys: 152 ms, total: 3.97 s
Wall time: 3.97 s


In [38]:
text = 'test test'

In [41]:
text[3:]

't test'

In [54]:
train_test = train_test.rename(columns={'0':0})

def detect_lang(col):
    try:
        shape = int(len(col)/2)
        col1 = col[:shape]
        col2 = col[shape:]
        return TextBlob(col1).detect_language(),TextBlob(col2).detect_language()
    except:
        return 'Unkmown'

train_test = train_test.reset_index(drop=True)
train_test= train_test[:100]
train_test['lang'] =train_test[0].apply(detect_lang)

In [55]:
train_test

,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
0,explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",0000997932d777bf,0,0,0,0,0,0,1,"(en, en)"
1,aww match background colour seemingly stick thank talk january utc,0,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",000103f0d9cfb60f,0,0,0,0,0,0,1,"(en, en)"
2,fair use rationale image wonju jpg thank upload image wonju jpg notice image page specify image use fair use explanation rationale use wikipedia article constitute fair use addition boilerplate fa...,0,"""\nFair use rationale for Image:Wonju.jpg\n\nThanks for uploading Image:Wonju.jpg. I notice the image page specifies that the image is being used under fair use but there is no explanation or rati...",0005300084f90edc,0,0,0,0,0,0,1,"(en, en)"
3,however moonlite edit note golden daph optus wake wikkis funny,0,"However, the Moonlite edit noted by golden daph was me (on optus ...) Wake up wikkis. So funny",003f698d06c9b180,0,0,0,0,0,0,1,"(en, en)"
4,rex mundi create stub rex mundi rex mundi high school thing know aunt donna bob griese go please add anything might know btw dad panther live princeton,0,Rex Mundi \n\nI've created a stub on Rex Mundi at Rex Mundi High School. Only thing I know about it is that both my Aunt Donna and Bob Griese went there. Please add anything you might know about...,02baab63953460ab,0,0,0,0,0,0,1,"(en, en)"
5,hi redrose apology delay draft propose present new section page volume ultimate allocation cover report allocation british railway steam diesel & electric locomotive br stock additional informatio...,0,Hi RedRose and apologies for delay. Here is a draft of what I propose to present on the 'new section' pages.\nThe 6 volumes of Ultimate Allocations cover the 'reported' allocations of all British ...,1a7d7c88372e5668,0,0,0,0,0,0,1,"(en, en)"
6,think meet notability guideline thank create page big icon pageant drag community drag gay community general death cover several mainstream medium outlet,0,I think she meets Notability guidelines and thanks for creating the page for her. She was a big icon in the pageant drag community and in the drag/gay community in general. Her death has been cove...,173558f5c2a8883b,0,0,0,0,0,0,1,"(en, en)"
7,nice tavern article article like joy wikipedia keep good work thank appreciate plan eat first time two week seek grace,0,"""\n\n Nice tavern article \n\nArticles like this are the joy of Wikipedia. Keep up the good work. \nThanks. I appreciate it. I plan to eat there for the first time in two weeks. Seek his grace """,17359f940f219872,0,0,0,0,0,0,1,"(en, en)"
8,mfd nomination user wikitrevor sandbox opinion matter welcome please participate discussion add comment wikipedia miscellany deletion user wikitrevor sandbox please sure sign comment four tilde ~~...,0,MfD nomination of User:Wikitrevor/Sandbox\n. Your opinions on the matter are welcome; please participate in the discussion by adding your comments at Wikipedia:Miscellany for deletion/User:Wikitre...,173693e634e1a6cf,0,0,0,0,0,0,1,"(en, en)"
9,speak sale speak percentage much hard obtain sale percentage say half people p resistance fall man gear war sell million copy xbox sell million copy gear war little fact speak,0,"I am not speaking of sales. I am speaking of percentage, much harder to obtain than sales, and that percentage says more than half of the people who own PS3's own Resistance: Fall of Man. Gears of...",17375e3fff2d9d1a,0,0,0,0,0,0,1,"(en, en)"


In [2]:
from textblob import TextBlob

b = TextBlob("bonjour")
b.detect_language()

train_test

train_test = train_test.rename(columns={'0':0})

def detect_lang(col):
    try:
        col1 = col[:len(col)/2]
        col2 = col[len(col/2):]
        return TextBlob(col1).detect_language(),TextBlob(col2).detect_language()
    except:
        return 'Unkmown'

train_test = train_test.reset_index(drop=True)

train_test= train_test[:1000]

%%time
train_test['lang'] =train_test[0].apply(detect_lang)

CPU times: user 3.81 s, sys: 140 ms, total: 3.95 s
Wall time: 3.95 s


In [31]:
train_test_lang = pd.read_csv('../output/train_test_lang.csv')

In [32]:
train_test_lang

,Unnamed: 0,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
0,0,explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",0000997932d777bf,0,0,0,0,0,0,1,Unkmown
1,1,aww match background colour seemingly stick thank talk january utc,0,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",000103f0d9cfb60f,0,0,0,0,0,0,1,Unkmown
2,2,fair use rationale image wonju jpg thank upload image wonju jpg notice image page specify image use fair use explanation rationale use wikipedia article constitute fair use addition boilerplate fa...,0,"""\nFair use rationale for Image:Wonju.jpg\n\nThanks for uploading Image:Wonju.jpg. I notice the image page specifies that the image is being used under fair use but there is no explanation or rati...",0005300084f90edc,0,0,0,0,0,0,1,Unkmown
3,3,however moonlite edit note golden daph optus wake wikkis funny,0,"However, the Moonlite edit noted by golden daph was me (on optus ...) Wake up wikkis. So funny",003f698d06c9b180,0,0,0,0,0,0,1,Unkmown
4,4,rex mundi create stub rex mundi rex mundi high school thing know aunt donna bob griese go please add anything might know btw dad panther live princeton,0,Rex Mundi \n\nI've created a stub on Rex Mundi at Rex Mundi High School. Only thing I know about it is that both my Aunt Donna and Bob Griese went there. Please add anything you might know about...,02baab63953460ab,0,0,0,0,0,0,1,Unkmown
5,5,hi redrose apology delay draft propose present new section page volume ultimate allocation cover report allocation british railway steam diesel & electric locomotive br stock additional informatio...,0,Hi RedRose and apologies for delay. Here is a draft of what I propose to present on the 'new section' pages.\nThe 6 volumes of Ultimate Allocations cover the 'reported' allocations of all British ...,1a7d7c88372e5668,0,0,0,0,0,0,1,Unkmown
6,6,think meet notability guideline thank create page big icon pageant drag community drag gay community general death cover several mainstream medium outlet,0,I think she meets Notability guidelines and thanks for creating the page for her. She was a big icon in the pageant drag community and in the drag/gay community in general. Her death has been cove...,173558f5c2a8883b,0,0,0,0,0,0,1,Unkmown
7,7,nice tavern article article like joy wikipedia keep good work thank appreciate plan eat first time two week seek grace,0,"""\n\n Nice tavern article \n\nArticles like this are the joy of Wikipedia. Keep up the good work. \nThanks. I appreciate it. I plan to eat there for the first time in two weeks. Seek his grace """,17359f940f219872,0,0,0,0,0,0,1,Unkmown
8,8,mfd nomination user wikitrevor sandbox opinion matter welcome please participate discussion add comment wikipedia miscellany deletion user wikitrevor sandbox please sure sign comment four tilde ~~...,0,MfD nomination of User:Wikitrevor/Sandbox\n. Your opinions on the matter are welcome; please participate in the discussion by adding your comments at Wikipedia:Miscellany for deletion/User:Wikitre...,173693e634e1a6cf,0,0,0,0,0,0,1,Unkmown
9,9,speak sale speak percentage much hard obtain sale percentage say half people p resistance fall man gear war sell million copy xbox sell million copy gear war little fact speak,0,"I am not speaking of sales. I am speaking of percentage, much harder to obtain than sales, and that percentage says more than half of the people who own PS3's own Resistance: Fall of Man. Gears of...",17375e3fff2d9d1a,0,0,0,0,0,0,1,Unkmown


In [30]:
train_test[train_test['lang']!='en']

,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
128,also 討論,0,There are also and ... (討論),17d174028a21f969,0,0,0,0,0,0,1,zh-TW
196,genealogy mv also likely ancestor gerda von siemens marry joachim von oertzen parent kristin von oertzen bear november liessow marry july munich civ july munich rel michael karl august wilhelm pri...,0,"Genealogy\nmv to here ...\n\nHe's also the likely ancestor of Gerda von Siemens, married to Joachim von Oertzen, the parents of: \n Kristin von Oertzen (born November 6 1944, Liessow), married on ...",183c52a3c38f1884,0,0,0,0,0,0,1,de
215,penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d~~penis d...,3,penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~penis 8===D~~p...,1a8afaad7cd0a4de,0,0,1,1,0,1,1,id
240,hitler duechebag,2,hitler was a duechebag,187aa645feeba1da,0,1,0,0,0,1,1,de
280,excellent work congratulation che dicono signa lo dicono signa,0,"""\nExcellent work - congratulations on both! -) Che dicono a Signa?Lo dicono a Signa. """,18ace69030fdef66,0,0,0,0,0,0,1,it
309,firśt aṃ ģoinģ ţo ţie ŷou uṗ keeṗ ŷou çonśçiouś durinģ ţhe folloŵinģ ṗroçeśś ţhrough ţhe uśe aṃṃonia nexţ ŵill ṃuţilaţe ŷour ģeniţalś forçe ŷou ţo eaţ ţheṃ ŵill çuţ biţś ŷour śkin ṃake ģlove ouţ ţ...,3,"Firśt, I aṃ ģoinģ ţo ţie ŷou uṗ and keeṗ ŷou çonśçiouś durinģ ţhe folloŵinģ ṗroçeśś ţhrough ţhe uśe of aṃṃonia. Nexţ, I ŵill ṃuţilaţe ŷour ģeniţalś and forçe ŷou ţo eaţ ţheṃ. Then I ŵill çuţ off b...",18d4d21e96049ee7,0,0,1,0,1,1,1,ro
386,hamsun centre article obituary prof atle kittang deletion ref article hamsun centre prof atle kittang cf edit summary sorry eisfbnore wrong wikipedia attribute pls edit war retrieve http en wikipe...,0,"""\n\nHamsun-centre, article on obituary by Prof. Atle Kittang\nRe deletion of ref. to article at Hamsun-centre by prof. Atle Kittang: Cf. edit-summary: """"Sorry, Eisfbnore, wrong on this being Wiki...",192df60afd1481aa,0,0,0,0,0,0,1,no
417,nonsense remember correct error zofia kossak szczucka say one novel write głośni krzyżowcy loud crusader nonsense novel title krzyżowcy crusader pole read novel,0,"""\n\nWhat nonsense!? As I remember I only corrected error in """"Zofia Kossak- Szczucka"""". It says that one of novels that she wrote is """"Głośni krzyżowcy"""" (Loud crusaders). THAT is nonsense. This ...",1952c70fce0764a1,0,0,0,0,0,0,1,pl
494,review dominique vidal le monde diplomatique,0,Review by Dominique Vidal in Le Monde Diplomatique,19b48c23ca089929,0,0,0,0,0,0,1,fr
873,,0,"[24 and 25, after 23]",1b6138b7f9fd87cb,0,0,0,0,0,0,1,Unkmown


In [7]:
collection = train_test[0]

In [ ]:
# try:
# %%time
bow_df= make_bow(collection)

train_test = train_test.reset_index(drop=True)
train_index = train_test[train_test['train_test']==1].index
test_index = train_test[train_test['train_test']==0].index

In [ ]:
w2v, model = make_w2v(collection)

# %%time
###подготовим датасеты
# bow_df = make_bow(collection)
# w2v_mean_df= make_w2v_mean(collection)
w2v_tfidf_df =  make_w2v_tfidf(collection)

artm_df = make_artm(collection)
print (bow_df.shape[0])
# print w2v_mean_df.shape[0]
print (w2v_tfidf_df.shape[0])
print (artm_df.shape[0])

list_clfs = [train_xgboost, train_lr]
data_sources = [bow_df, w2v_tfidf_df, artm_df]

In [9]:
send_to_telegram('препроцессинг сделан')

In [ ]:
    # list_clfs = [train_xgboost]
    # data_sources = [bow_df]

    # bow_df = pd.DataFrame(bow_df.toarray())

    # %%time
try:    
    preds_all=pd.DataFrame()
    targets = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    for target in targets:
        print (target)
        df_y = train_test.iloc[train_index][target]
    #     artm_df = make_artm(collection)
    #     list_clfs = [train_lr]
    #     data_sources = [l]

        for num_source, source in enumerate(data_sources):
            print (num_source)
            train = source.iloc[train_index]
            y = train_test.iloc[train_index][target]
            test = source.iloc[test_index]
            for num_clf, i in enumerate(list_clfs):
                if (num_source == 0) & (num_clf == 1):
                    print ('пропускаем')
                    pass
                elif (num_source == 1) & (num_clf == 0):
                    print ('пропускаем')
                    pass
                elif (num_source == 2) & (num_clf == 0):
                    print ('пропускаем')
                    pass
                else:
                    print (i)
                    preds = i(train, y, test, 'probability')
                    preds = pd.DataFrame(preds)
                    preds['clf']=str(num_source) + ' ' + str(num_clf)
                    preds_all = pd.concat([preds_all, preds], axis=1)
    send_to_telegram('все сделано')
    # preds_all = mix_predictions(preds_all, ['0 0', '1 1', '2 1'])
except:
    send_to_telegram('ошибка')
    

In [12]:
preds_all

,0,1,clf,0,1,clf,0,1,clf,0,1,clf,0,1,clf,0,1,clf
0,0.100558,0.899442,0 0,0.711928,0.288072,0 0,0.045177,0.954823,0 0,0.988628,0.011372,0 0,0.169111,0.830889,0 0,0.980951,0.019049,0 0
1,0.934357,0.065643,0 0,0.996083,0.003917,0 0,0.977331,0.022669,0 0,0.998331,0.001669,0 0,0.972583,0.027417,0 0,0.995568,0.004432,0 0
2,0.934357,0.065643,0 0,0.996083,0.003917,0 0,0.977331,0.022669,0 0,0.998331,0.001669,0 0,0.972583,0.027417,0 0,0.995568,0.004432,0 0
3,0.942423,0.057577,0 0,0.996083,0.003917,0 0,0.978942,0.021058,0 0,0.997892,0.002108,0 0,0.974703,0.025297,0 0,0.996272,0.003728,0 0
4,0.934357,0.065643,0 0,0.996083,0.003917,0 0,0.977331,0.022669,0 0,0.998331,0.001669,0 0,0.972583,0.027417,0 0,0.995568,0.004432,0 0
5,0.947335,0.052665,0 0,0.996760,0.003240,0 0,0.980907,0.019093,0 0,0.998485,0.001515,0 0,0.980593,0.019407,0 0,0.996666,0.003334,0 0
6,0.938609,0.061391,0 0,0.996083,0.003917,0 0,0.979065,0.020935,0 0,0.998331,0.001669,0 0,0.974529,0.025471,0 0,0.995976,0.004024,0 0
7,0.934357,0.065643,0 0,0.996083,0.003917,0 0,0.977331,0.022669,0 0,0.998331,0.001669,0 0,0.972583,0.027417,0 0,0.995568,0.004432,0 0
8,0.905417,0.094583,0 0,0.997006,0.002994,0 0,0.978935,0.021065,0 0,0.998331,0.001669,0 0,0.949039,0.050961,0 0,0.995568,0.004432,0 0
9,0.960451,0.039549,0 0,0.998023,0.001977,0 0,0.985950,0.014050,0 0,0.998620,0.001380,0 0,0.982911,0.017089,0 0,0.995976,0.004024,0 0


In [14]:
sub = pd.read_csv('sample_submission.csv')
submission = pd.concat([sub[['id']], preds_all[1]],axis=1)

In [15]:
submission.columns = ('id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate')

In [16]:
submission.to_csv('sub_xgb_20170402.csv', index=False)

In [17]:
!zip -r sub_xgb_20170402.csv.zip sub_xgb_20170402.csv

  adding: sub_xgb_20170402.csv (deflated 72%)


In [2]:
import os
from functools import reduce
sub_dir = '../input/toxic/subs/'

files=[]
for i in os.listdir(sub_dir):
    files.append(sub_dir+ i)

preds = [pd.read_csv(f, index_col='id') for f in files]
pred = reduce(lambda a, b: a + b, preds) / len(preds)
pred = pred.reset_index()
pred.to_csv('avg_20180204_v4.csv', index=False)

In [3]:
import pandas as pd

In [4]:
pd.read_json('preds_all.json')

""


In [3]:
pred.columns

Index(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [4]:
!zip -r avg_20180204_v4.csv.zip avg_20180204_v4.csv

  adding: avg_20180204_v4.csv (deflated 53%)


In [5]:
import telebot

In [15]:
bot = telegram.Bot(token='451841380:AAFLpx5Tm8f-3VEGx2QEPEz3EAeQXwnjG0U')

In [27]:
API_TOKEN = '451841380:AAFLpx5Tm8f-3VEGx2QEPEz3EAeQXwnjG0U'

bot = telebot.TeleBot(API_TOKEN)

In [21]:
chat_id = -1001289775579

In [32]:
import telegram, telegram.ext

tb = telegram.ext.Updater(API_TOKEN)
def a(bot, update):
    bot.sendMessage(update.message.chat_id, "responding to /start cmd")

tb.dispatcher.add_handler(telegram.ext.CommandHandler("start", '@like'))
tb.start_polling()

In [25]:
mes_id= bot.send_message(chat_id, 'text').message_id

In [26]:
mes_id

20

In [13]:
bot.send_message?

In [60]:
pd.read_csv('../input/toxic/subs/sub_xgb_20170402.csv')

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.899442,0.288072,0.954823,0.011372,0.830889,0.019049
1,0000247867823ef7,0.065643,0.003917,0.022669,0.001669,0.027417,0.004432
2,00013b17ad220c46,0.065643,0.003917,0.022669,0.001669,0.027417,0.004432
3,00017563c3f7919a,0.057577,0.003917,0.021058,0.002108,0.025297,0.003728
4,00017695ad8997eb,0.065643,0.003917,0.022669,0.001669,0.027417,0.004432
5,0001ea8717f6de06,0.052665,0.003240,0.019093,0.001515,0.019407,0.003334
6,00024115d4cbde0f,0.061391,0.003917,0.020935,0.001669,0.025471,0.004024
7,000247e83dcc1211,0.065643,0.003917,0.022669,0.001669,0.027417,0.004432
8,00025358d4737918,0.094583,0.002994,0.021065,0.001669,0.050961,0.004432
9,00026d1092fe71cc,0.039549,0.001977,0.014050,0.001380,0.017089,0.004024


In [59]:
pd.read_csv('avg_20180204_v2.csv')

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.940762,0.121477,0.885401,0.011653,0.789122,0.146591
1,0000247867823ef7,0.022374,0.001329,0.007615,0.000573,0.009754,0.001505
2,00013b17ad220c46,0.030329,0.001523,0.008395,0.000571,0.010147,0.001539
3,00017563c3f7919a,0.019815,0.001413,0.007404,0.000779,0.009998,0.001292
4,00017695ad8997eb,0.030734,0.002201,0.011200,0.001136,0.014522,0.002129
5,0001ea8717f6de06,0.027348,0.001327,0.016376,0.000613,0.012931,0.002232
6,00024115d4cbde0f,0.023477,0.001816,0.008456,0.000775,0.010309,0.001763
7,000247e83dcc1211,0.673194,0.020067,0.474438,0.005500,0.529431,0.024974
8,00025358d4737918,0.038172,0.002167,0.011581,0.000958,0.020744,0.002447
9,00026d1092fe71cc,0.015385,0.000748,0.005890,0.000660,0.006728,0.001505


In [53]:
sub[:2]

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5


In [58]:
pred.reset_index()[:2]

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.940762,0.121477,0.885401,0.011653,0.789122,0.146591
1,0000247867823ef7,0.022374,0.001329,0.007615,0.000573,0.009754,0.001505


In [55]:
submission[:2]

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.899442,0.288072,0.954823,0.011372,0.830889,0.019049
1,0000247867823ef7,0.065643,0.003917,0.022669,0.001669,0.027417,0.004432


In [161]:
preds_all[1]

,1,1,1,1,1,1
0,0.997310,0.007635,0.991895,0.001654,0.977123,0.349294
1,0.000058,0.000045,0.000049,0.000043,0.000048,0.000044
2,0.000058,0.000045,0.000049,0.000043,0.000048,0.000044
3,0.000058,0.000045,0.000049,0.000043,0.000048,0.000044
4,0.000058,0.000045,0.000049,0.000043,0.000048,0.000044
5,0.000058,0.000045,0.000049,0.000043,0.000048,0.000044
6,0.000058,0.000045,0.000049,0.000043,0.000048,0.000044
7,0.999394,0.000045,0.998856,0.000043,0.998792,0.000044
8,0.000252,0.000045,0.000063,0.000043,0.000069,0.000044
9,0.000058,0.000045,0.000049,0.000043,0.000048,0.000044


In [136]:
preds_all['clf'].unique()

array(['0 0'], dtype=object)

In [142]:
test_size = sub.shape[0]

In [137]:
preds_all.shape

(918984, 3)

In [141]:
num_parts = preds_all.shape[0]/sub.shape[0]

In [144]:
preds_all['check'] = 0

In [150]:
preds_all.index.max()

153163

In [155]:
new_sub = pd.DataFrame()
for _ in range(int(num_parts)):
    part = preds_all[:test_size]
    preds_all = preds_all[test_size:]
    new_sub = pd.concat([new_sub, part],axis=1)
    
    

In [156]:
new_sub

,0,1,clf,check,0,1,clf,check,0,1,...,clf,check,0,1,clf,check,0,1,clf,check


In [95]:
# preds_all = mix_predictions(preds_all, ['0 0', '1 0'])
#
source_test=read_data_test(test_path)###for general test
# source_test = y_true.reset_index(drop=True)#for validation

sub = make_submission(preds_all, source_test)
sub = sub[['max_value0']]
#parsed_date_money = pd.read_excel('/root/projects/output/promiss/parsed_date_money.xlsx')
parsed_date_money = pd.read_json('/root/projects/output/promiss/parsed_date_money.json')
parsed_date_money = pd.concat([parsed_date_money, sub], axis=1)
#parsed_date_money.to_excel('/root/projects/output/promiss/parsed_date_money_classification.xlsx')
parsed_date_money.to_json('/root/projects/output/promiss/parsed_date_money_classification.json')

NameError: name 'read_data_test' is not defined

In [7]:
temp_data = pd.read_csv('../input/toxic/train_test_temp.csv')

In [15]:
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 200
temp_data[['comment_text', '0']]

,comment_text,0
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",aww match background colour seemingly stick thank talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about...",hey man really try edit war guy constantly remove relevant information talk edit instead talk page seem care format actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tid...",make real suggestion improvement wonder section statistic late subsection type accident think reference may need tidy exact format ie date format etc late one else first preference format style re...
4,"You, sir, are my hero. Any chance you remember what page that's on?",sir hero chance remember page
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,congratulation well use tool well · talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,cocksucker piss around work
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",vandalism matt shirvington article revert please ban
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encycl...",sorry word nonsense offensive anyway intend write anything article wow would jump vandalism merely request encyclopedic one use school reference selective breed page almost stub point animal breed...
9,alignment on this subject and which are contrary to those of DuLithgow,alignment subject contrary dulithgow


In [17]:
import pandas as pd

data = '{"id":{"0":411083201,"1":418513660,"2":528057543,"3":586713622,"4":647656728,"5":647656785,"6":1493456455,"7":1493456487,"8":2005894602,"9":2095344770,"10":2187987262,"11":2411692096,"12":2411698457,"13":2474058013,"14":2474058022,"15":2881830804,"16":2895323815,"17":2895323816,"18":2983919102,"19":3321734312,"20":3641568148,"21":4010355532,"22":4030622426,"23":4037746568,"24":4055719117,"25":4259001279,"26":4340535594,"27":4625994189,"28":4666687025},"Latitude":{"0":37.7673162,"1":37.7645003,"2":37.7682118,"3":37.7648492,"4":37.771672,"5":37.7721693,"6":37.7763591,"7":37.777046,"8":37.7690716,"9":37.766319,"10":37.7664253,"11":37.7770377,"12":37.7763106,"13":37.7739857,"14":37.774748,"15":37.775488,"16":37.7752166,"17":37.7759142,"18":37.7744588,"19":37.7760172,"20":37.7762395,"21":37.765011,"22":37.769195,"23":37.7750452,"24":37.7726713,"25":37.7717782,"26":37.7745253,"27":37.7768943,"28":37.7752822},"Longitude":{"0":-122.4219479,"1":-122.4216812,"2":-122.4223857,"3":-122.4320119,"4":-122.4331366,"5":-122.4307254,"6":-122.4180877,"7":-122.4172737,"8":-122.4277243,"9":-122.417422,"10":-122.4290387,"11":-122.4175698,"12":-122.4232558,"13":-122.424226,"14":-122.4226877,"15":-122.4159,"16":-122.4195185,"17":-122.4191912,"18":-122.4205881,"19":-122.4314951,"20":-122.4168763,"21":-122.4226685,"22":-122.4315398,"23":-122.4210561,"24":-122.4220741,"25":-122.4167145,"26":-122.4306476,"27":-122.4245402,"28":-122.4161381},"Cafe Name":{"0":"Four Barrel Coffee","1":"Muddy Waters","2":"Carlin\'s Cafe","3":"Peet\'s Coffee & Tea","4":"Nectar","5":"Cafe International","6":"Ma\'velous","7":"Starbucks","8":"Starbucks","9":"Flying Pig Bistro","10":"Church Street Cafe","11":"Anderson Bakery","12":"Blue Bottle Coffee","13":"mercury cafe","14":"gourmet and more","15":"Cumaica","16":"All Star Cafe","17":"Boston Cafe","18":"Javalencia Cafe","19":"Alamo Square Cafe","20":"Blue Bottle Coffee","21":"Stanza","22":"Duboce Park Cafe","23":"Eden Cafe","24":"Delessio Market & Bakery","25":"Gaslamp Cafe","26":"The Center SF Tea House","27":"Cafe la Vie","28":"Peet\'s Coffee"},"Street":{"0":"Valencia Street","1":"Valencia Street","2":"Valencia Street","3":"Market Street","4":"Haight Street","5":"Haight Street","6":"Market Street","7":"Market Street","8":"Market Street","9":"South Van Ness Avenue","10":null,"11":null,"12":null,"13":"Octavia Street","14":null,"15":"Mission Street","16":"Market Street","17":"Van Ness Avenue","18":"Market Street","19":null,"20":"Market Street","21":null,"22":"Sanchez Street","23":"Franklin Street","24":"Market Street","25":"Howard Street","26":"Fillmore Street","27":"Octavia Street","28":null}}'

df = pd.read_json(data)

df.head()
location = df['Latitude'].mean(), df['Longitude'].mean()
import folium

locationlist = df[["Latitude","Longitude"]].values.tolist()
labels = df["Cafe Name"].values.tolist()

m = folium.Map(location=location, zoom_start=14)
for point in range(len(locationlist)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(locationlist[point], popup=popup).add_to(m)

m